# Directions for use of Python scripts
To encourage uptake of the methods described in this study, we have provided all of the Python code necessary to implement detection of scent-marking detection in accelerometer data, and link those detected events to GPS data collected concurrently. Some devices may produce data files that follow a different format, but the code here should be general enough to implement for most devices with some minor adjustment to the code. In this notebook, we describe the sequential steps of the workflow. In steps that require third party software, we provide links to where this software can be downloaded (all are freely available) and will endevour to make sure these links are updated when software is hosted elsewhere. For steps that require use of our Python code, we provide the code itself in the cell that follows each stage description. 

1. When using the AX3 accelerometer, use OMGui (https://github.com/digitalinteraction/openmovement/wiki/AX3-GUI) to convert the .cwa files in to a resampled CSV file. We down-sampled data to 25 Hz to improve computation speed.
    
2.	Do a quick summary plot using the crop_file.py. We used this script to crop the file, as the device continued to record after it was removed from the dogs, producing large files with superfluous data. Use the ‘chunksize’ argument to specify how many rows should be taken from the start of the file. The code in crop_file.py ;

In [ ]:
import pandas as pd
import os
chunksize = 10*10**4 #change here to set number of rows to load from start of file

os.chdir('C:\\DataFolder\...')
filename = '' #file name in folder here

chunk = pd.read_csv(filename, nrows =chunksize)

#Convert time column to datetime format    
chunk['Time'] = pd.to_datetime(chunk['Time'])

##Look at last time in chunk
chunk.tail(1)[3]


#plot axes to see if whole deployment is present
time = chunk.iloc[:,0]
x = chunk.iloc[:,1]
y = chunk.iloc[:,2]
z = chunk.iloc[:,3]

#plot just last 100 rows to see if the device is still attached
import matplotlib.pyplot as plt
plt.plot(time.tail(100), y.tail(100), color='lightblue', linewidth=3)
plt.show()

#shows we have the right file size
#now write to csv
chunk.to_csv('') # pick a filename for output

3.	Before we can use the KNN to predict scent-marking events, we need to use an initial period of observation to train the model. In our study, we obtained a training set by observing the dog prior to release. Thus, the accelerometer file must be split in to training and testing periods. The next few steps detail how one may prepare and train the KNN.

4.	We must synchronize the timestamp of the accelerometer data with the video that details to behavioural observations of the dog. At the beginning of the observation period, we performed conspicuous calibration tilts (the device was rotated 90 degrees for 5 second, and this was repeated 3 times). With this in mind, use subsample.py to plot 10-minute summaries of the data. The function will run through the first 3 days of the file (after that the tag was off the animal and so it's just excess data), showing a 10 minute window in each plot to help you find the calibration tilts mentioned previously. This function needs there to be a folder called 'explore', which will be filled with plots for you to run through sequentially (using Windows’ Photo app for instance). In future, we will implement a more elegant interactive plot for Users to use in Python, but this is currently in development. Once the tilts are identified and their time in the accelerometer file noted, you can trim the start of the file to ca. 5 seconds before the tilts, to make them easy to spot and sync to the video in ELAN. The code for subsample.py ;

In [ ]:
import pandas as pd
import os

os.chdir('C:\\DataFolder\...') #Data location here
filename = '' #filename from crop_file.py
dog_df = pd.read_csv(filename, usecols = ['Time', 'Accel-X (g)', ' Accel-Y (g)', ' Accel-Z (g)']) #Use only specific columns, this arguement can be changed based on format produced by device
dog_df.set_index('Time', inplace=True) #set the index to time column

twentyHz = dog_df.iloc[1::2,:] #take every 2nd row to downsample (halves the sample rate)

twentyHz.to_csv('dog_dfmine_Twenty.csv') #save a copy of downsampled data

##Plot in matplotlib the first 1000 rows
import matplotlib.pyplot as plt
plt.plot(twentyHz.head(1000), color='lightblue', linewidth=3) 
plt.show()

#####################################################
#Search for calibration tilts
# code needs an output folder, called 'explore'
path = 'C:\\DataFolder\explore'
try:  
    os.mkdir(path)
except OSError:  
    print ("Creation of the directory %s failed" % path)
else:  
    print ("Successfully created the directory %s " % path)
	
#Here we can use the datetime record in our accl data, and the time the device was started, to produce an 'elapsed seconds' column
twentyHz['seconds'] = (pd.to_datetime(twentyHz.index) - pd.to_datetime(pd.DataFrame({'year': [2018],'month': [6],'day': [13], 'hour':[9], 'minute':[36], 'second':[27.525]}))[0]).total_seconds()
twentyHz.set_index(twentyHz['seconds'], inplace=True) #change the index to seconds elapsed
twentyHz['minutes'] = twentyHz['seconds']/60
twentyHz.set_index(twentyHz['minutes'], inplace=True)

#For this example, I made plots for a range of 0 to 1440 (1440 minutes is 24 hours)
#With that, I found the tilts between 132 and 134 minutes in to the file
#Your data may differ

for counter in range(0,1440):
    start = counter*1
    stop = start + 10
    fig = twentyHz[(twentyHz['minutes'] > start) & (twentyHz['minutes'] < stop)].iloc[:,0:3].plot(figsize=(20,10)).get_figure()
    dir_name = 'C:\\Data\Hopland Dogs\dog_dfmine\explore'
    base_filename = '{0}{1}'.format('output',counter)
    suffix = '.jpg'
    fig.savefig(os.path.join(dir_name, base_filename + suffix))
    plt.close(fig)
    #input("Press Enter to continue...")

#Load back in the dog_df 20Hz data
twentyHz = pd.read_csv('dog_df_Twenty.csv')
twentyHz.set_index('Time', inplace=True)
twentyHz['seconds'] = (pd.to_datetime(twentyHz.index) - pd.to_datetime(pd.DataFrame({'year': [2018],'month': [6],'day': [13], 'hour':[9], 'minute':[36], 'second':[27.525]}))[0]).total_seconds()
twentyHz['minutes'] = twentyHz['seconds']/60
   
#To write this one, I used the 'minutes' column, the tilts start ~27 minutes in to the data   
start = 27
stop = start + 2
twentyHz[(twentyHz['minutes'] > start) & (twentyHz['minutes'] < stop)].iloc[:,0:3].plot(figsize=(20,10)) #are tilts visible? 

cut = twentyHz[(twentyHz['minutes'] > 27)] #cut the file to just before the calibration tilts
cut.head(1000).iloc[:,0:3].plot() #check with a quick plot

#set seconds to start from 0
cut.iloc[0,]
cut['seconds'] = cut['seconds']-1620.040000
cut['minutes'] = cut['minutes']-27.000667
cut.iloc[0:1000,0:3].plot(figsize=(20,10))

cut.to_csv('dog_df_Cut.csv') #write to file, for loading in to ELAN
print('Finished writing')


5.	Load the trimmed data in to ELAN, along with the video. Sync the two using the procedure detailed in the video tutorial kindly produced by Cassim Ladha (https://www.youtube.com/watch?v=zofLvUU0Gus). Then run through the video and make annotations. Following this, output annotations as a tab delimited text file to load back in to Python.

6.	Using the label_accl.py script, load the accelerometer data and the annotation record, combine the two to produce one annotated accelerometer dataset. This will output an accelerometer file with an extra column that contains the class of behaviour being undertaken by the animal at that time. The code for label_accl.py ;

In [ ]:
# This script is to annotate the accl data using the annotation record from ELAN
import pandas as pd
import os

# Load accl data and set time to start from the first calibration point
os.chdir('C:\\DataFolder\...') #Data location here
filename = 'dog_df_Cut.csv' #the output from subsample.py
twentyHz = pd.read_csv(filename)

twentyHz.set_index('seconds', inplace=True)

# In this example, ELAN said the offset was 21 seconds, so plot to make sure and then cut
start = 20.6
stop = start + 40
twentyHz[(twentyHz.index > start) & (twentyHz.index < stop)].plot(figsize=(10, 5))

# from annotation, we can see the video ends at 31 minutes, 1.653 seconds
# that's 1861.653 seconds long

# For later use, create csv for the unobserved period that begins immmediately at the end of the video
df_unobs = twentyHz[(twentyHz.index > 1861.653)]
df_unobs.iloc[::20, :].iloc[:,0:4].plot() #quickly plot every 20th row to get a summary of file


#I will pass all data through KNN and then cut the end according to the timestamp
df_unobs.to_csv('dog_unobserved.csv') #a file that contains only unobserved data, from free living deployments etc.

# cut another file to duration of video for KNN training and validation
twentyHz = twentyHz[(twentyHz.index > start) & (twentyHz.index < 1861.653)]
# set start time to 0
twentyHz.index = twentyHz.index - twentyHz.index[0]

# now load in annotations
annot = pd.read_table('dog_Annotate.txt', delimiter="\t", header=None)
annot.columns = ['class', 'nans', 'start', 'end', 'duration', 'text'] #the columns typically output from ELAN
annot.start.iloc[0] = 0.0

# add column to put our annotations
import numpy as np

twentyHz['class'] = np.nan #fill with NaN for now

#The following code takes the behavioural record from annot and uses them to populate the 'class' column in twentyHz
import pandas as pd
import numpy as np
import time

bins = np.unique(annot[['start', 'end']].values.flatten())  # get time range bins
annot['period'] = '['+annot['start'].astype(str) + ', ' + annot['end'].astype(str) + ')'
categorized = pd.cut(twentyHz.index, bins, right=False).astype(str) # categorize into bins based on time, right=False means start is inclusive, end is exclusive
categorized = pd.Series(categorized).to_frame('labels') # make it a dataframe to get ready for merge below
twentyHz['class'] = categorized.merge(annot, left_on='labels', right_on='['+annot['start'].round(decimals=3).astype(str) + ', ' + annot['end'].round(decimals=3).astype(str) + ')', how='left')['class'].fillna('no class').values # merge with processed annot frame to get the correct labels


twentyHz.to_csv('dog_annot_accl.csv') #this accelerometer file now has behavioural annotations for use in the KNN

print('Script Finished')

7.	Use the KNN_sm.py script to actually train and test the KNN classifier to the annotated data. Functions to obtain performance metrics (e.g. Accuracy) are included, along with a means to construct confusion matrices to evaluate the KNN’s performance. KNN_sm.py code;

In [ ]:
import pandas as pd
import os
os.chdir('C:\\DataFolder\...') #Data location here

#Load in annotated accl data
accl_data = pd.read_csv('dog_annot_accl.csv') #from label_accl.py
accl_data.set_index('seconds', inplace=True) #set the index

#remove all calibration stuff, and data that wasn't labelled at the start of the file
accl_data = accl_data[accl_data['class'] != 'calibration']

#set x (accl data) and y (classes)
x = accl_data.iloc[:,1:4]
y = accl_data['class']

#Split accl_data to testing and training sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42, shuffle=True)

#check training and testing have the same classes in them
set(y_train) == set(y_test)

## Import the KNN Classifier.
from sklearn.neighbors import KNeighborsClassifier
## Start the KNN instance, setting k to 5 neighbors. 
knn = KNeighborsClassifier(n_neighbors=5)
## Fit the model on the training data.
knn.fit(X_train, y_train.values.ravel())
## See how the model performs on the test data.
knn.score(X_test, y_test) #This is the accuracy

##Get Array of all predictions
y_pred = knn.predict(X_test)

##Calculate confusion matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import cohen_kappa_score

confusion_matrix(y_test, y_pred)
accuracy_score(y_test, y_pred) #Accuracy
cohen_kappa_score(y_test, y_pred) #Cohen's Kappa

##Plot a confusion matrix
import itertools
import numpy as np
import matplotlib.pyplot as plt

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

##Get confusion matrix    
cnf_matrix = confusion_matrix(y_test, y_pred)
class_names = y_test.unique()

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names,
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')

plt.show()

##Build classification report
from sklearn.metrics import classification_report
#target_names = ['class 1', 'class 2', 'class 3', 'class 4']
print(classification_report(y_test, y_pred, target_names=class_names))

##Join x_test and y_pred for output
output = X_test
output['class'] = y_pred
output.sort_index(inplace = True)
output.to_csv('dog_pred_output.csv')

#plot output
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

# Data for plotting
t = np.arange(0.0, 2.0, 0.01)
s = 1 + np.sin(2 * np.pi * t)

# Let's plot predicted activity and actual activity
#Predicted
fig, ax = plt.subplots()
ax.plot(output.index, output['class'], 'o')

ax.set(xlabel='time (s)', ylabel='behavior',
       title='Predicted Activity - Nate')
ax.grid()
plt.show()
#fig.savefig("predicted.png")

#Actual
fig, ax = plt.subplots()
ax.plot(accl_data.index, accl_data['class'], 'o')

ax.set(xlabel='time (s)', ylabel='behavior',
       title='Actual Activity - Nate')
ax.grid()
plt.show()
#fig.savefig("actual.png")

#perform a full prediction
accl_data['pred_class'] = knn.predict(accl_data.iloc[:,1:4])
accl_data
knn.score(accl_data.iloc[:,1:4], accl_data['class']) #92.6%
accl_data.to_csv('dog_actual_pred.csv')


#Here we predict activity during the unobserved period (from full deployment etc.) 
#using the KNN that has been trained and validated above
dog_unobs = pd.read_csv('dog_unobserved.csv') #from label_accl.py script
dog_unobs.set_index('seconds', inplace=True)
dog_unobs.head(10)
dog_unobs['pred_class'] = knn.predict(dog_unobs.iloc[:,1:4])
dog_unobs.head(10)
dog_unobs.to_csv('dog_unobs_predicted.csv')

#KNN finished, now use smooth_knn.py to prepare predictions for matching with GPS data

8.	Use the smooth_knn.py script to smooth the KNN predictions to 1 Hz so that they can be linked to GPS locations recorded at that time. This script takes the 25 Hz accelerometer data and smooths it to 1 Hz, by taking the modal class for each 25-row group that constitutes a second of accelerometer data. There is also a function in this script to check each class' accuracy and ensure the smoothed data matches the actual classes as recorded by video (after those annotations too has been smoothed). The smooth_knn.py code;

In [ ]:
import pandas as pd
import numpy as np
import os
os.chdir('C:\\DataFolder\...') #Data location here

#Load in annotated accl data
accl_data = pd.read_csv('dog_actual_pred.csv') #data with actual and predicted classes from behavioural validation period 
accl_data.set_index('seconds', inplace=True)
accl_data['r_secs'] = np.around(accl_data.index, decimals = 0) #define the second epoch across which all sub-second data will be smoothed

#produce a downsampled (to 1 Hz) dataset, take the modal class from each second
smooth = pd.DataFrame(index = np.around(accl_data.index, decimals = 0).unique())
smooth['actual'] = accl_data.groupby('r_secs')['class'].agg(lambda x:x.value_counts().index[0])
smooth['pred'] = accl_data.groupby('r_secs')['pred_class'].agg(lambda x:x.value_counts().index[0])


# Let's plot predicted activity and actual activity
#Predicted
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
ax.plot(smooth.index, smooth['pred'], 'o')

ax.set(xlabel='time (s)', ylabel='behavior',
       title='Predicted Activity - Dog')
ax.grid()
plt.show()
#fig.savefig("predicted.png")

#Actual
fig, ax = plt.subplots()
ax.plot(smooth.index, smooth['actual'], 'o')

ax.set(xlabel='time (s)', ylabel='behavior',
       title='Actual Activity - Dog')
ax.grid()
plt.show()
#Looks pretty good, let's make sure our scent marks are being detected decently

#run quick accuracy check
def check(x, y):
    if len(x) == len(y):
        x = np.array(x)
        y = np.array(y)
        xy = np.where(y == x, 1, 0)
    else:
        dfx = pd.DataFrame(x)
        dfy = pd.DataFrame(y)
        df = dfx.merge(dfy, how='outer', on='seconds')
        df['cor'] = np.where(df.pred == df.actual, 1, 0)
        xy = df['cor']
    return np.mean(xy)

check(smooth['actual'],smooth['pred']) # 95.5% accuracy on the whole

#Can perform behaviour specific accuracy calculations here if you wish
#check(smooth['pred'][smooth.pred == 'right'],smooth['actual'][smooth.actual == 'right']) # 
#check(smooth['pred'][smooth.pred == 'left'],smooth['actual'][smooth.actual == 'left']) # 
#check(smooth['pred'][smooth.pred == 'scent'],smooth['actual'][smooth.actual == 'scent']) # 85.7% accuracy for left
#check(smooth['pred'][smooth.pred == 'lie'],smooth['actual'][smooth.actual == 'lie']) #99%
#check(smooth['pred'][smooth.pred == 'stand'],smooth['actual'][smooth.actual == 'stand']) #77%

#Check accuracy, seems pretty good, export smooth data ready to be synced to GPS
smooth.to_csv('dog_smooth_accl.csv')

#Run smoothing over the unobserved period predictions too
#Load in annotated accl data
dog_unobs = pd.read_csv('dog_unobs_predicted.csv') #file containing unobserved predictions, from KNN_sm.py
dog_unobs.set_index('seconds', inplace=True)
dog_unobs.head(10)
dog_unobs['r_secs'] = np.around(jas_unobs.index, decimals = 0)
dog_unobs.head(10)

#produce a downsampled (to 1 Hz) dataset, take the modal class from each second
smooth_unobs = pd.DataFrame(index = np.around(dog_unobs.index, decimals = 0).unique())
smooth_unobs['pred'] = dog_unobs.groupby('r_secs')['pred_class'].agg(lambda x:x.value_counts().index[0])
smooth_unobs.head(10)
smooth_unobs.to_csv('dog_unobs_smooth_preds.csv') #write out the 1 Hz predictions, ready to be linked to GPS data

9.	Using the link_gps.py script, merge the predicted classes of the accelerometer data (a product of the KNN) to the Latitude and Longitude values in the GPS data. Once the GPS data has behavioural classes, identify the points where dogs are making left and right scent marks. These points then can be written to file for use in GIS software or R for further analysis.

In [ ]:
import pandas as pd
import numpy as np
import os
os.chdir('C:\\DataFolder\...') #Data location here

#Load in annotated accl data
smooth = pd.read_csv('dog_smooth_accl.csv') #smoothed predictions from smooth_knn.py
smooth.set_index('seconds', inplace=True)

#change seconds in to actual date and time
#Note for this dog, 445 seconds on accelerometer time is 2018-06-13 10:11:13.165

#Here's some code explaining my method
#this line will define a datetime that described actual time at 445 seconds
pd.to_datetime(pd.DataFrame({'year': [2018],'month': [6],'day': [13], 'hour':[10], 'minute':[11], 'second':[13.165]}))[0]

#make an array that describes the time difference (in datetime) using the index values in seconds
pd.to_timedelta(smooth.index, 's')
smooth.tail(10)
pd.to_timedelta(pd.Series(range(0,1841)), 's')


#define a datetime column by adding the timedelta to the origin
smooth['datetime'] = pd.to_datetime(pd.DataFrame({'year': [2018],'month': [6],'day': [13], 'hour':[10], 'minute':[11], 'second':[13]}))[0] + pd.to_timedelta(pd.Series(range(0,1842)), 's')

#now smooth['datetime'] will describe the actual time, which can be linked to the GPS timestamp

#Now do the same for the unobserved data
dog_unobs = pd.read_csv('dog_unobs_smooth_preds.csv') #smoothed predictions for the unobserved period, from smooth_knn.py
dog_unobs.set_index('seconds', inplace=True)
dog_unobs['datetime'] = pd.to_datetime(pd.DataFrame({'year': [2018],'month': [6],'day': [13], 'hour':[10], 'minute':[11], 'second':[13]}))[0] + pd.to_timedelta(dog_unobs.index, 's')

#a few lines here to check everything is as it should be
dog_unobs.columns
dog_unobs['Time']
dog_unobs['datetime']
#check that they match up
smooth.head(10)
smooth.tail(10)
dog_unobs.head(10) #looks good!

#Load in GPS data
fields = ['Date', ' Time', ' Latitude', ' Longitude'] #which columns to take from CSV file, many devices produce additional columns, NOTE: the IgotU device introduced a space before some column names i.e. " Time"
gps = pd.read_csv('Dog_GPS.csv', usecols=fields) #Your file name may differ, define it here
gps.columns = ['Date', 'Time', 'Latitude', 'Longitude'] #Remove those spaces from column names
gps.set_index(pd.to_datetime(gps.iloc[:,0] + ' ' + gps.iloc[:,1]), inplace=True)

#cut GPS data to observed validation period start/end
start = smooth.datetime.min()
end = smooth.datetime.max()
cut_gps = gps[(gps.index >= start) & (gps.index <= end)]

#merge labels to gps data
smooth_gps = smooth.merge(cut_gps, left_on=smooth.datetime, right_on=cut_gps.index, how='left')
smooth_gps.to_csv('dog_gps_labeled.csv') #a file that contains all KNN predictions and LAT/LON coordinates for the observed validation period

smooth_gps[smooth_gps['pred'] == 'scent'] #how many scents?

#Sometimes, the GPS may not be recording at the exact second the accelerometer predicts a scent mark. 
#The code below will find the nearest gps pos available and annotate, also recording the time difference to give an idea of location confidence/error 
#this function finds the nearest datetime in 'items' to the datetime 'pivot'
def nearest(items, pivot):
    return min(items, key=lambda x: abs(x - pivot))

nearest(cut_gps.index, smooth[smooth['pred'] == 'scent'].iloc[0].datetime) #nearest was at 2018-05-11 14:59:47

#right = smooth_gps[(smooth_gps.key_0 == nearest(cut_gps.index, smooth[smooth['pred'] == 'right'].iloc[0].datetime))]
#left = smooth_gps[(smooth_gps.key_0 == nearest(cut_gps.index, smooth[smooth['pred'] == 'left'].iloc[0].datetime))]
scent = smooth_gps[(smooth_gps.key_0 == nearest(cut_gps.index, smooth[smooth['pred'] == 'scent'].iloc[0].datetime))]

#Annotate the GPS data with the unobserved predictions too
#Load in GPS data files
fields = ['Date', ' Time', ' Latitude', ' Longitude']
gps = pd.read_csv('Dog_GPS.csv', usecols=fields)
gps.columns
gps.columns = ['Date', 'Time', 'Latitude', 'Longitude']
gps['datetime'] = pd.to_datetime(gps.iloc[:,0] + ' ' + gps.iloc[:,1])
gps.set_index('datetime', inplace=True)

#cut GPS data to unobserved period
start = dog_unobs.datetime.min()
end = dog_unobs.datetime.max()
unobs_gps_only = gps[(gps.index >= start) & (gps.index <= end)]

#merge labels to gps data
unobs_gps = dog_unobs.merge(unobs_gps_only, left_on=dog_unobs.datetime, right_on=unobs_gps_only.index, how='left')
unobs_gps = unobs_gps.iloc[:,[1,2,5,6]]
unobs_gps.set_index('datetime', inplace=True)

unobs_gps[(unobs_gps['pred'] == 'scent')] #how many scent-marks?
unobs_gps['dt'] = unobs_gps.index
#not all scent marks had gps during event, so let's find nearest

for i in range(len(scent_marks)):
    nrest = nearest(unobs_gps_only.index, unobs_gps[(unobs_gps['pred'] == 'scent')].dt.iloc[i])
    scent_marks.iloc[i,4] = pd.to_timedelta(scent_marks.iloc[i].name-nrest, 's').total_seconds()
    scent_marks.iloc[i,1] = unobs_gps.Latitude[(unobs_gps.index == nrest)].values
    scent_marks.iloc[i,2] = unobs_gps.Longitude[(unobs_gps.index == nrest)].values

scent_marks.to_csv('Dog_unobs_scentmarks.csv') #This file describes all of the scent-mark times and locations from the unobserved (deployment) period

#This code in case you need all behavioural types
#unobs_gps = unobs_gps[unobs_gps.Latitude.notnull()]
#unobs_gps.to_csv('gps_unobs_labelled.csv')

#Personally, I prefer the mapping tools available in R and QGIS.
#If you would prefer to map these locations in Python, here is some simple code that you can extend for your purposes

#plot left and right scent marks on a map
from gmplot import gmplot
smooth_gps.columns

#center map
gmap = gmplot.GoogleMapPlotter(scent_marks['Latitude'].mean(), scent_marks['Longitude'].mean(), 13) #lat, log, zoom
gmap.coloricon = "http://www.googlemapsmarkers.com/v1/%s/"
#take left and right scent mark coords and plot
points = scent_marks
gmap.scatter(points.Latitude.values, points.Longitude.values, color='#3B0B39', size=20)
gmap.draw('scentmarks.html')

#make a map of all locations
gps_all = smooth_gps[np.isfinite(smooth_gps.Latitude)]

gmap = gmplot.GoogleMapPlotter(smooth_gps['Latitude'].mean(), smooth_gps['Longitude'].mean(), 13) #lat, log, zoom
gmap.coloricon = "http://www.googlemapsmarkers.com/v1/%s/"
#take left and right scent mark coords and plot
gmap.heatmap(gps_all.Latitude.values, gps_all.Longitude.values) #makes a heatmap of point locations
gmap.scatter(points.Latitude.values, points.Longitude.values, color='#3B0B39', size=2, marker = False)
gmap.draw('allpoints.html')

10.	These steps can then be repeated using data from the unobserved period (obtained during deployment on free living animals for instance) to obtain a record of scent-marking locations. The necessary code segments are presented in the scripts above too.